In [1]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json
import statistics as stat
from collections import deque

In [1]:
# READ IN DATA AS PANDAS DATAFRAME
filename = 'data/MEBL3/MEBL3_Pressure-data-2024-06-28 09_40_08.csv'
pressure_df = pd.read_csv(filename, parse_dates=['Time']).fillna(0)    
print(f"{filename} : file read into a pandas dataframe.")

pressure_df

NameError: name 'pd' is not defined

In [3]:
# CONVERT TO NUMPY ARRAY
pressure_arr = pressure_df.to_numpy()
pressure_arr

array([[Timestamp('2024-06-14 01:55:29'), 1.85e-07, 5.55e-10],
       [Timestamp('2024-06-14 01:55:30'), 1.84e-07, 5.67e-10],
       [Timestamp('2024-06-14 01:55:31'), 1.83e-07, 5.54e-10],
       ...,
       [Timestamp('2024-06-27 23:59:56'), 1.64e-07, 3.41e-10],
       [Timestamp('2024-06-27 23:59:57'), 1.65e-07, 3.38e-10],
       [Timestamp('2024-06-27 23:59:58'), 1.64e-07, 3.4e-10]],
      dtype=object)

In [4]:
x = np.array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
             [3, 2, 5, 1, 8, 4, 0, 9, 6, 7]])
x

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [3, 2, 5, 1, 8, 4, 0, 9, 6, 7]])

In [5]:
queue = []
for row_idx in range(len(x[0])):
    queue.append(x[1][row_idx])
    if len(queue) < 5:
        continue
    elif len(queue) > 5:
        queue.pop(0)
    print(queue)
    local_mean = stat.median(queue)
    print(f"{local_mean=}")

[3, 2, 5, 1, 8]
local_mean=3
[2, 5, 1, 8, 4]
local_mean=4
[5, 1, 8, 4, 0]
local_mean=4
[1, 8, 4, 0, 9]
local_mean=4
[8, 4, 0, 9, 6]
local_mean=6
[4, 0, 9, 6, 7]
local_mean=6


In [ ]:
# PRESSURE SPIKE SCRIPT 1
## strung out because ~10x slower than script 2
'''
for type_idx, type in enumerate(["chamber", "column"]):
    pressure_deque = []
    spike_times = []
    for row_idx in range(len(pressure_arr)): #O(n)
        pressure_value = pressure_arr[row_idx][type_idx+1]
        pressure_deque.append(pressure_value)
        if len(pressure_deque) < 5:
            continue
        elif len(pressure_deque) > 5:
            pressure_deque.pop(0) # don't know runtime of list.pop(), but size of pressure_queue is at most 6 so negligible
        local_mean = np.mean(pressure_deque) # np.mean > stat.mean for speed but not precision
        if pressure_value > 2*local_mean:
            spike_time = str(pressure_arr[row_idx][0])
            print(f"{spike_time=} : {pressure_value}")
            print(f"{pressure_deque=}")
            spike_times.append(spike_time)
    with open(f"data/pressure_spike_times/{type}_spike_times.json", "w") as outfile: 
        json.dump(spike_times, outfile)
'''
# 5 seconds

In [9]:
# PRESSURE SPIKE SCRIPT 2
## difference: running tally of sum, manually calculate mean from rolling sum (no use of np.mean())
time_range = 5
for type_idx, type in enumerate(["chamber", "column"]):
    spike_times = []
    pressure_deque = deque(maxlen=5)
    sum_pressure = 0
    for row_idx in range(len(pressure_arr)):
        pressure_value = pressure_arr[row_idx][type_idx + 1]
        queue_len = len(pressure_deque)
        if queue_len == 5:
            sum_pressure -= pressure_deque.popleft()
        pressure_deque.append(pressure_value)
        sum_pressure += pressure_value
        if queue_len < 5:
            continue
        local_mean = sum_pressure / 5
        if pressure_value > 2 * local_mean:
            spike_time = str(pressure_arr[row_idx][0])
            print(f"{spike_time=} : {pressure_value}")
            spike_times.append(spike_time)
    with open(f"data/pressure_spike_times/{type}_spike_times.json", "w") as outfile:
        json.dump(spike_times, outfile)

### QUESTION, ARE PRESSURE SPIKES DEFINED BY JUST THE INCREASE OR WILL THERE ALSO BE A SHARP DECREASE AFTERWARDS
### May need to make rolling window threshold computation apply to the middle value in the deque
### Also, 0.6 seconds ?!?!

spike_time='2024-06-15 07:02:55' : 7e-07
spike_time='2024-06-17 07:03:28' : 1.04e-08
spike_time='2024-06-17 16:45:07' : 2.68e-09
spike_time='2024-06-22 18:50:10' : 4.37e-09
spike_time='2024-06-23 06:34:54' : 2.36e-09
spike_time='2024-06-23 06:35:05' : 2.8e-08
spike_time='2024-06-23 06:35:31' : 2.08e-09
spike_time='2024-06-23 07:06:12' : 1.54e-09
spike_time='2024-06-25 01:44:55' : 1.85e-08
spike_time='2024-06-26 18:52:32' : 1.54e-09
